## #다음 testfile1.txt, testfile2.txt, testfile3.txt에 대해서 질문에 답하시오. 
1. testfile1.txt 에 대해서 각 사람별 평균을 구하시오.
2. testfile2.txt, testfile3.txt 에 공통적으로 나타나는 단어에 대해 빈도수를 내림차순으로 출력하시오. 

In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("local").appName("week5-lab").getOrCreate()

In [3]:
tf2 = spark_session.sparkContext.textFile("./data/testfile2.txt")
tf3 = spark_session.sparkContext.textFile("./data/testfile3.txt")
print(tf2.take(5))
print(tf3.take(2))

["pairRDD = sc.parallelize( [('panda', 0), ('pink', 3), ('pirate', 3), ('panda', 1), ('pink', 4)] )", 'result1 = pairRDD.groupByKey()', 'result2 = result1.map(lambda x : (x[0], len(x[1])))', 'result3 = result1.map(lambda x : (x[0], list(x[1])))', '']
['sc = SparkContext(conf=conf)', '']


In [48]:
import sys, re

wordcounts = tf2.filter(lambda line: len(line) > 0) \
			   .flatMap(lambda line: re.split('\W+', line)) \
			   .filter(lambda word: len(word) > 1) \
                .map(lambda word:(word.lower(),1)) \
			   .reduceByKey(lambda v1, v2: v1 + v2) \
			   .persist()

                
print(wordcounts.take(10))

[('pairrdd', 2), ('sc', 1), ('parallelize', 1), ('panda', 2), ('pink', 2), ('pirate', 1), ('result1', 3), ('groupbykey', 1), ('result2', 2), ('map', 2)]


In [44]:
t3rdd_wordcnt = tf3.filter(lambda line: len(line) > 0) \
			   .flatMap(lambda line: re.split('\W+', line)) \
			   .filter(lambda word: len(word) > 1) \
			   .map(lambda word:(word.lower(),1)) \
			   .reduceByKey(lambda v1, v2: v1 + v2) \
			   .persist()
t3rdd_wordcnt.take(5)

[('sc', 3),
 ('sparkcontext', 1),
 ('conf', 2),
 ('pairrdd1', 2),
 ('parallelize', 2)]

In [51]:
common=wordcounts.join(t3rdd_wordcnt).mapValues(lambda x: x[0]+x[1]).sortBy(lambda x: x[1], ascending=False)
print(common.collect())

[('result1', 5), ('sc', 4), ('panda', 4), ('pink', 4), ('parallelize', 3), ('print', 3), ('collect', 3), ('pirate', 2)]
